### 筆記
* annotation version: tumor as kidney(DATA_0811_2)
* INPUT_SIZE = 331 (INPUT_SIZE = 512電腦跑不動)
* EPOCHS = 100
* BATCH_SIZE = 2 (BATCH_SIZE = 4 電腦跑不動)
* 有在Encoder中載入用ImageNet預訓練的模型參數 (keras.applications規定輸入維度必須是(331, 331, 3)才能載入用ImageNet預訓練的模型參數)

### 讀取函式庫、超參數設定

In [1]:
### 讀取函式庫 ###
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

In [2]:
import tensorflow as tf
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


In [3]:
dataset_dir = 'DATA_0811_2\\WW600WL100'
x_train_path = 'DATA_0811_2\\WW600WL100\\train\\images'
y_train_path = 'DATA_0811_2\\WW600WL100\\train\\annotations_tumor as kidney'
checkpoints_path = 'DATA_0811_2\WW600WL100\\Model_NASNetLarge-Unet_0906'
model_name = 'NASNetLarge-Unet'
start_from_latest_checkpoints = False

INPUT_SIZE = 331 # input size equal to output size
CLASS_NUM = 2 # background and roi(kidney)

# initialize the number of epochs and batch size
EPOCHS = 100
BATCH_SIZE = 2

IMAGE_ORDERING = 'channels_last' # 不要亂改這個參數

### 用生成器(generator)生成訓練圖片
使用generator的好處在於，模型在訓練(或驗證)的過程中會依據批量大小生成圖片，不需要在程式中預先準備好資料集(通常資料集都會大到無法一口氣塞入記憶體之中)，減少記憶體的使用空間。另外，使用ImageDataGenerator可以對生成的圖片實施「資料增強(Data Augmentation)」，增加模型的泛化能力。

In [4]:
### 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑) ###
def get_pairs_from_paths(images_path, segs_path, ignore_non_matching=False):
    """ Find all the images from the images_path directory and
        the segmentation images from the segs_path directory
        while checking integrity of data """

    ACCEPTABLE_IMAGE_FORMATS = [".jpg", ".jpeg", ".png", ".bmp"]
    ACCEPTABLE_SEGMENTATION_FORMATS = [".png", ".bmp"]

    image_files = []
    segmentation_files = {}

    for dir_entry in os.listdir(images_path):
        if os.path.isfile(os.path.join(images_path, dir_entry)) and \
                os.path.splitext(dir_entry)[1] in ACCEPTABLE_IMAGE_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            image_files.append((file_name, file_extension,
                                os.path.join(images_path, dir_entry)))

    for dir_entry in os.listdir(segs_path):
        if os.path.isfile(os.path.join(segs_path, dir_entry)) and \
           os.path.splitext(dir_entry)[1] in ACCEPTABLE_SEGMENTATION_FORMATS:
            file_name, file_extension = os.path.splitext(dir_entry)
            full_dir_entry = os.path.join(segs_path, dir_entry)
            if file_name in segmentation_files:
                raise DataLoaderError("Segmentation file with filename {0}"
                                      " already exists and is ambiguous to"
                                      " resolve with path {1}."
                                      " Please remove or rename the latter."
                                      .format(file_name, full_dir_entry))

            segmentation_files[file_name] = (file_extension, full_dir_entry)

    return_value = []
    # Match the images and segmentations
    for image_file, _, image_full_path in image_files:
        if image_file in segmentation_files:
            return_value.append((image_full_path,
                                segmentation_files[image_file][1]))
        elif ignore_non_matching:
            continue
        else:
            # Error out
            raise DataLoaderError("No corresponding segmentation "
                                  "found for image {0}."
                                  .format(image_full_path))

    return return_value

### 將CT影像的陣列轉成適合模型輸入的形式(維度轉換 + 標準化) ###
def get_image_array(image_input,
                    width, height,
                    imgNorm="sub_mean", ordering='channels_first'):
    """ Load image array from input """

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_image_array: path {0} doesn't exist"
                                  .format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_image_array: Can't process input type {0}"
                              .format(str(type(image_input))))

    if imgNorm == "sub_and_divide": # 除以127.5，然後減 1
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean": # 減去ImageNet的平均BGR
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide": # 除以255
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img/255.0

    if ordering == 'channels_first':
        img = np.rollaxis(img, 2, 0)
    return img

### 將標記資料的陣列轉成適合模型輸入的形式(維度轉換) ###
def get_segmentation_array(image_input, nClasses,
                           width, height, no_reshape=False):
    """ Load segmentation array from input """

    seg_labels = np.zeros((height, width, nClasses))

    if type(image_input) is np.ndarray:
        # It is already an array, use it as it is
        img = image_input
    elif isinstance(image_input, six.string_types):
        if not os.path.isfile(image_input):
            raise DataLoaderError("get_segmentation_array: "
                                  "path {0} doesn't exist".format(image_input))
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_segmentation_array: "
                              "Can't process input type {0}"
                              .format(str(type(image_input))))

    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST)
    img = img[:, :, 0]

    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c).astype(int)

    if not no_reshape:
        seg_labels = np.reshape(seg_labels, (width*height, nClasses))

    return seg_labels

import itertools
import random
from keras.preprocessing.image import ImageDataGenerator

def image_segmentation_generator(images_path, segs_path, batch_size,
                                 n_classes, input_height, input_width,
                                 output_height, output_width, do_augment=False):

    img_seg_pairs = get_pairs_from_paths(images_path, segs_path) 
        # 取得資料集中所有的檔案路徑(包含CT影像和其對應標記的路徑)
    random.shuffle(img_seg_pairs) # 打散檔案路徑
    zipped = itertools.cycle(img_seg_pairs) 
        # 將檔案路徑用循環迭代器(iterator)封裝；範例：cycle('ABCD') --> A B C D A B C D ...
#     counter = 0

    while True:
        # 如果模型訓練完一輪訓練集中所有的資料，就重新打散檔案路徑
#         if counter >= len(img_seg_pairs):
#             counter = 0
#             random.shuffle(img_seg_pairs)
#             zipped = itertools.cycle(img_seg_pairs)             
        
        X = []
        Y = []
        for _ in range(batch_size): # batch_size多大，就取得多少份資料
#             counter += 1
            
            im, seg = next(zipped) # 取得CT影像和其對應標記的路徑

            im = cv2.imread(im, 1) # 1 = cv2.IMREAD_COLOR (讀取彩色圖片)
            seg = cv2.imread(seg, 1)
            
            if do_augment:
                ### Example of transforming images and masks together. ###
                # we create two instances with the same arguments
                data_gen_args = dict(featurewise_center=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         featurewise_std_normalization=False, # 範例程式碼為True，但這裡我只是要把一張圖片變成是增強後的型態
                         #rotation_range=30, # 範例程式碼為90
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.2)
                image_datagen = ImageDataGenerator(**data_gen_args)
                mask_datagen = ImageDataGenerator(**data_gen_args)
                # Provide the same seed and keyword arguments to the fit and flow methods
                seed = random.randint(0,10000)
                im_itr = image_datagen.flow(im.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                seg_itr = mask_datagen.flow(seg.reshape(1, 512, 512, 3), batch_size=1, seed=seed)  
                im = next(im_itr).reshape(512, 512, 3)
                seg = next(seg_itr).reshape(512, 512, 3)
            
            # 將CT影像和其對應的標記轉換成模型輸入的形式
            X.append(get_image_array(im, input_width,
                                     input_height, imgNorm="sub_mean", ordering=IMAGE_ORDERING)) 
                # imgNorm預設為sub_mean"，但這裡我改用圖片最常實施的正規化方法(同除以255)
            Y.append(get_segmentation_array(
                seg, n_classes, output_width, output_height))

        yield np.array(X), np.array(Y)

Using TensorFlow backend.


In [5]:
# 建立訓練資料的生成器
train_gen = image_segmentation_generator(images_path = x_train_path, segs_path = y_train_path, batch_size = BATCH_SIZE, 
                                         n_classes = CLASS_NUM, input_height = INPUT_SIZE, input_width = INPUT_SIZE, 
                                         output_height = INPUT_SIZE, output_width = INPUT_SIZE, do_augment=False)

### 設定回調函式(callbacks)
keras_segmentation自定義的回調函式好像缺了什麼，直接用在這份程式碼中並不會正常運作，因此我直接用keras.callbacks.ModelCheckpoint定期儲存模型參數。

In [6]:
from keras.callbacks import ModelCheckpoint
callbacks = [ModelCheckpoint(os.path.join(checkpoints_path, model_name + '_{epoch}.h5'), 
                             save_weights_only=True, 
                             period=5)] # period=5：每5個Epoch才會儲存一次參數

### 建立模型

In [7]:
import keras
from keras.models import *
from keras.layers import *
from keras import layers

#### Eecoder - NASNetLarge

In [8]:
from keras.applications import NASNetLarge
input_height = input_width = INPUT_SIZE
#model_ = NASNetLarge(input_shape=(INPUT_SIZE,INPUT_SIZE,3), include_top=False, weights=None)
model_ = NASNetLarge(weights='imagenet')
model_.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [9]:
def one_side_pad(x):
    x = ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING)(x)
    if IMAGE_ORDERING == 'channels_first':
        x = Lambda(lambda x: x[:, :, :-1, :-1])(x)
    elif IMAGE_ORDERING == 'channels_last':
        x = Lambda(lambda x: x[:, :-1, :-1, :])(x)
    return x

img_input = model_.get_layer('input_1').output
#f0 = one_side_pad(model_.get_layer('stem_conv1').output) # 需要把維度從(255,255)調整成(256,256)才能在之後concatenate
f0 = model_.get_layer('stem_conv1').output
f1 = model_.get_layer('concatenate_1').output
f2 = model_.get_layer('normal_concat_5').output
f3 = model_.get_layer('normal_concat_12').output
f4 = model_.get_layer('normal_concat_18').output

#### Decoder (這裡我有為了讓輸出大小和輸入大小保持一致新增含有上取樣層的block)

In [10]:
if IMAGE_ORDERING == 'channels_first':
    MERGE_AXIS = 1
elif IMAGE_ORDERING == 'channels_last':
    MERGE_AXIS = -1

o = f4

o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(512, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = Lambda(lambda o: o[:, :-1, :-1, :])(o) # 不取最右邊的一欄(維度調整)
o = (concatenate([o, f3], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(256, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = (concatenate([o, f2], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
#o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (Conv2D(128, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = Lambda(lambda o: o[:, :-1, :-1, :])(o) # 不取最右邊的一欄(維度調整)
o = (concatenate([o, f1], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
#o = (Conv2D(32, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = Lambda(lambda o: o[:, :-1, :-1, :])(o) # 不取最右邊的一欄(維度調整)
o = (concatenate([o, f0], axis=MERGE_AXIS))
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)

### 增加一個block讓圖片的輸出大小等於輸出大小 ###
o = (UpSampling2D((2, 2), data_format=IMAGE_ORDERING))(o)
o = one_side_pad(o) # 在最右邊新增補零的一欄(維度調整)
o = (ZeroPadding2D((1, 1), data_format=IMAGE_ORDERING))(o)
o = (Conv2D(64, (3, 3), padding='valid', data_format=IMAGE_ORDERING))(o)
o = (BatchNormalization())(o)
###

o = Conv2D(CLASS_NUM, (3, 3), padding='same', data_format=IMAGE_ORDERING)(o) # 輸出層

### 模仿 keras_segmentation 在輸出層之後額外添加這幾層 ###
if IMAGE_ORDERING == 'channels_first':
    o = (Reshape((CLASS_NUM, INPUT_SIZE * INPUT_SIZE)))(o)
    o = (Permute((2, 1)))(o)
elif IMAGE_ORDERING == 'channels_last':
    o = (Reshape((INPUT_SIZE * INPUT_SIZE, CLASS_NUM)))(o)
o = (Activation('softmax'))(o)
###

In [11]:
model = Model(img_input, o)

#### 查看模型架構

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 165, 165, 96) 0           stem_bn1[0][0]                   
____________________________________________________________________________________________

#### 載入模型參數(optional)

In [13]:
# model.load_weights(os.path.join(checkpoints_path, model_name + '_' + EPOCHS + '.h5'))

### 編譯模型

In [14]:
from keras.optimizers import Adam, Adadelta
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(lr = 1.0),
              metrics = ['accuracy'])

import keras.backend as K
print(K.eval(model.optimizer.lr)) # 確保是1.0

1.0


### 訓練模型

In [15]:
st = time.time()
print(f'########## 即將用該資料集進行訓練：{dataset_dir} ##########')

if start_from_latest_checkpoints:
    try:
        model.load_weights(
            os.path.join(
                checkpoints_path, 
                model_name + '_' + str(max([int(i.split('_')[-1][:-3]) for i in os.listdir(checkpoints_path)])) + '.h5'
            )
        )
    except:
        print('Model weights not found!')

if not os.path.exists(checkpoints_path):
    os.makedirs(checkpoints_path)
    print('-----建立新資料夾：' + checkpoints_path + '-----') 


model.fit_generator(
    train_gen,
    steps_per_epoch = 512, 
    #steps_per_epoch = len(x_train) // BATCH_SIZE,
    epochs = EPOCHS,
    verbose = 1,
    callbacks=callbacks
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

########## 即將用該資料集進行訓練：DATA_0811_2\WW600WL100 ##########
Epoch 1/100
512/512 [==============================] - 425s 829ms/step - loss: 0.1127 - accuracy: 0.9681
Epoch 2/100
512/512 [==============================] - 356s 696ms/step - loss: 0.0102 - accuracy: 0.9967
Epoch 3/100
512/512 [==============================] - 355s 694ms/step - loss: 0.0068 - accuracy: 0.9977
Epoch 4/100
512/512 [==============================] - 354s 692ms/step - loss: 0.0048 - accuracy: 0.9984
Epoch 5/100
512/512 [==============================] - 356s 695ms/step - loss: 0.0057 - accuracy: 0.9982
Epoch 6/100
512/512 [==============================] - 355s 694ms/step - loss: 0.0046 - accuracy: 0.9985
Epoch 7/100
512/512 [==============================] - 354s 692ms/step - loss: 0.0044 - accuracy: 0.9986
Epoch 8/100
512/512 [==============================] - 357s 697ms/step - loss: 0.0037 - accuracy: 0.9987
Epoch 9/100
512/512 [==============================] - 354s 692ms/step - loss: 0.0029 - accuracy: 0.999

512/512 [==============================] - 355s 694ms/step - loss: 9.9106e-04 - accuracy: 0.9996
Epoch 78/100
512/512 [==============================] - 356s 695ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 79/100
512/512 [==============================] - 356s 695ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 80/100
512/512 [==============================] - 354s 692ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 81/100
512/512 [==============================] - 354s 690ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 82/100
512/512 [==============================] - 355s 694ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 83/100
512/512 [==============================] - 355s 694ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 84/100
512/512 [==============================] - 355s 693ms/step - loss: 0.0010 - accuracy: 0.9996
Epoch 85/100
512/512 [==============================] - 356s 695ms/step - loss: 9.7286e-04 - accuracy: 0.9996
Epoch 86/100
512/512 [=============================

### 模型預測結果(測試集)

In [16]:
from evaluate import *
preds = predict_from_folder(model, 'DATA_0811_2\\WW600WL100\\test\\images', INPUT_SIZE, 2)

100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [17:16<00:00, 12.21it/s]


In [17]:
st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
    checkpoints_path = None,
    calculate_predicting_indicators = True,
    output_predicted_result = False, 
    segment_out_predicted_region_from_original_images = False, 
    roi_description = 'tumor as kidney', 
    preds = preds)

print('訓練集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
print('')
print('混淆矩陣：')
print(f'True Positive: {result[-4]}')
print(f'False Positive: {result[-3]}')
print(f'False Negative: {result[-2]}')
print(f'True Negative: {result[-1]}')

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------開始計算各項預測指標----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
訓練集預測結果：
average Dice score per case of kidney:  0.9523
average recall of kidney:  0.9439
average precision of kidney:  0.9617
global dice score of kidney:  0.9472

混淆矩陣：
True Positive: 4051
False Positive: 132
False Negative: 68
True Negative: 8405
花費時間(秒)：690.7522349357605


In [18]:
f = open("DATA_0811_2\\patient indices of testing set - KiTS.txt", "r")
test_patient_idx = f.read().splitlines()
f.close()

dice_score_list = result[5]
print('測試集各病患的 Dice score:')
for idx, i in enumerate(dice_score_list):
    print(f'case{test_patient_idx[idx]}: {i: .4f}')

測試集各病患的 Dice score:
case150:  0.9653
case151:  0.8723
case152:  0.9660
case153:  0.9711
case154:  0.9409
case155:  0.9404
case156:  0.9529
case157:  0.9527
case158:  0.9664
case159:  0.9510
case160:  0.9525
case161:  0.9493
case162:  0.9611
case163:  0.9639
case164:  0.9546
case165:  0.9550
case166:  0.9627
case167:  0.9706
case168:  0.9487
case169:  0.9652
case170:  0.9337
case171:  0.9664
case172:  0.9524
case173:  0.9546
case174:  0.9612
case175:  0.9620
case176:  0.9419
case177:  0.9396
case178:  0.8937
case179:  0.9652
case181:  0.9620
case182:  0.9601
case183:  0.9487
case184:  0.9257
case185:  0.9656
case186:  0.9732
case187:  0.9309
case188:  0.9453
case189:  0.9317
case190:  0.9449
case191:  0.9650
case192:  0.9572
case193:  0.9622
case194:  0.9317
case195:  0.9603
case196:  0.9668
case197:  0.9393
case198:  0.9595
case199:  0.9547
case200:  0.9533
case201:  0.9705
case202:  0.9573
case204:  0.9645
case205:  0.9521
case206:  0.9375
case207:  0.9663
case208:  0.9497
case209:  0

In [19]:
# st = time.time()

# from evaluate import *
# _ = evaluate_model(
#     image_dir = os.path.join(dataset_dir, 'test', 'images'), 
#     label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
#     checkpoints_path = None,
#     calculate_predicting_indicators = False,
#     output_predicted_result = True, 
#     segment_out_predicted_region_from_original_images = True, 
#     roi_description = 'tumor as kidney', 
#     preds = preds
# )

# ed = time.time()
# spend_time = ed - st
# print('花費時間(秒)：' + str(spend_time))

In [20]:
# show_result(
#     target_dataset_base_dir = 'DATA_0811_2\\WW600WL100\\test',
#     result_num = 10,
#     roi_description = 'tumor as kidney', 
#     roi_name_chinese = '腎臟',
#     show_predicted_result = True,
#     show_segmentation_result = True, 
#     image_scale = 4)